# Introduction

Let's start with a example of an Alpine API session.

1. Initialize a session.
1. Take a tour some commands.
1. Run a workflow and download the results.

Import the Python Alpine API and some other useful packages.

In [1]:
import alpine as AlpineAPI

In [2]:
from pprint import pprint
import json

### Setup

Have access to a workflow on your Alpine instance that you can run. You'll need a few pieces of information in order to login and run the workflow. First, find the url of the open workflow. It should look something like:

`https://<AlpineHost>:<PortNum>/#workflows/<WorkflowID>`

You'll also need your Alpine username and password.

I've stored my connection information in a configuration file named `alpine_login.conf` that looks something like this:
```JSON
    {
        "host": "AlpineHost",
        "port": "PortNum",
        "username": "fakename",
        "password": "12345"
    }
```

In [3]:
filename = "alpine_login.conf"

with open(filename, "r") as f:
    data = f.read()

conn_info = json.loads(data)

host = conn_info["host"]
port = conn_info["port"]
username = conn_info["username"]
password = conn_info["password"]

Here are the names of a workspace and a workflow within it that we want to run.

In [4]:
test_workspace_name = "API Sample Workspace"
test_workflow_name = "Data ETL"

Create a session and login the user.

In [5]:
session = AlpineAPI.APIClient(host, port, username, password)

### Use the API

Get information about the Alpine instance.

In [6]:
pprint(session.get_license())

{u'admins': 400,
 u'advisor_now_enabled': True,
 u'analytics_developer': 100,
 u'analytics_developer_limit_reached': False,
 u'branding': u'alpine',
 u'business_user': 100,
 u'business_user_limit_reached': False,
 u'client_name': None,
 u'collaborator': 100,
 u'collaborator_limit_reached': False,
 u'correct_mac_address': True,
 u'data_analyst': 100,
 u'data_analyst_limit_reached': False,
 u'expired': False,
 u'expires': u'2099-12-31',
 u'is_enabled_api': True,
 u'is_enabled_custom': True,
 u'is_enabled_jobs': True,
 u'is_enabled_milestones': True,
 u'is_enabled_modeling': True,
 u'is_enabled_touchpoints': True,
 u'level': None,
 u'limit_api': False,
 u'limit_custom': False,
 u'limit_jobs': False,
 u'limit_milestones': False,
 u'limit_modeling': False,
 u'limit_touchpoints': False,
 u'mac_address': u'!!!!!!!!!!',
 u'organization_uuid': None,
 u'users_license_limit_reached': False,
 u'vendor': u'alpine',
 u'version': u'6.3.0.0.5410-4ef43d3c9\n',
 u'workflow_enabled': True}


In [7]:
pprint(session.get_version())

u'6.3.0.0.5410-4ef43d3c9'


Find information about the logged-in user.

In [8]:
pprint(session.get_status())

{u'admin': True,
 u'auth_method': u'internal',
 u'dept': u'',
 u'email': u'tjbay@alpinenow.com',
 u'entity_type': u'user',
 u'first_name': u'T.J.',
 u'id': 7,
 u'image': {u'complete_json': True,
            u'entity_type': u'image',
            u'icon': u'/users/7/image?style=icon&1482194432',
            u'original': u'/users/7/image?style=original&1482194432'},
 u'is_deleted': None,
 u'last_name': u'Bay',
 u'ldap_group_id': None,
 u'notes': u'',
 u'roles': [u'admin'],
 u'subscribed_to_emails': False,
 u'tags': [],
 u'title': u'Assistant Regional Manager',
 u'user_type': u'analytics_developer',
 u'username': u'tjbay',
 u'using_default_image': True}


Find information on all users.

In [9]:
len(session.user.get_list())

103

Find my user ID and then use it to update my user data.

In [10]:
user_id = session.user.get_id(username)

In [11]:
pprint(session.user.update(user_id, title = "Assistant to the Regional Manager"))

{u'admin': True,
 u'auth_method': u'internal',
 u'complete_json': True,
 u'dept': u'',
 u'email': u'tjbay@alpinenow.com',
 u'entity_type': u'user',
 u'first_name': u'T.J.',
 u'id': 7,
 u'image': {u'complete_json': True,
            u'entity_type': u'image',
            u'icon': u'/users/7/image?style=icon&1482194432',
            u'original': u'/users/7/image?style=original&1482194432'},
 u'is_deleted': None,
 u'last_name': u'Bay',
 u'ldap_group_id': None,
 u'notes': u'',
 u'roles': [u'admin'],
 u'subscribed_to_emails': False,
 u'tags': [],
 u'title': u'Assistant to the Regional Manager',
 u'user_type': u'analytics_developer',
 u'username': u'tjbay',
 u'using_default_image': True}


A similar set of commands can be used to create and update workspaces and the membership of each workspace.

In [12]:
test_workspace_id = session.workspace.get_id(test_workspace_name)
session.workspace.member.add(test_workspace_id, user_id);

### Run a workflow

To run a workflow use the Process subclass of the Workfile class. The `wait_until_finished` method will periodically query the status of the running workflow and returns control to the user when the workflow has completed.

In [13]:
workflow_id = session.workfile.get_id(workfile_name = "Data ETL",
                                      workspace_id = test_workspace_id)

process_id = session.workfile.process.run(workflow_id)

session.workfile.process.wait_until_finished(workflow_id = workflow_id,
                                             process_id = process_id,
                                             verbose = True,
                                             query_time = 5)

Workflow in progress for ~88.1 seconds.                 


u'SUCCESS'

We can download results using the `download_results` method. The flow results contain a summary of the output of each operator as well as metadata about the flow run.

In [14]:
flow_results = session.workfile.process.download_results(workflow_id, process_id)
pprint(flow_results, depth=2)

{u'flowMetaInfo': {u'endTime': u'2017-04-25T15:00:35.178-0700',
                   u'executeUser': u'7',
                   u'noOfError': 0,
                   u'noOfNodesProcessed': 3,
                   u'processId': u'4b820a92-08d1-49f7-9aac-ce07eef4dd3d',
                   u'startTime': u'2017-04-25T14:59:03.259-0700',
                   u'status': u'SUCCESS',
                   u'workflowId': u'1269',
                   u'workflowName': u'Data ETL'},
 u'logs': [{...}, {...}, {...}, {...}, {...}, {...}, {...}],
 u'outputs': [{...}, {...}, {...}]}
